In [1]:
!pip install medspacy

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
import spacy
import medspacy
from medspacy.sentence_splitting import PyRuSHSentencizer
import os
from subprocess import Popen, PIPE, STDOUT
import pandas as pd

text_path = '../../multicardioner_train+dev+test+bg_240502/track2/cardioccc_dev/en/brat'
annotations_path = '../../multicardioner_train+dev+test+bg_240502/track2/cardioccc_dev/en/brat'
output_path = '../../multicardioner_train+dev+test+bg_240502/track2/cardioccc_dev/en/sentences' 
sents_per_batch = 1 # max number of sentences in each output file
is_test = False

In [4]:
nlp = spacy.blank("en")

In [5]:
nlp.add_pipe("medspacy_pyrush")

In [12]:
import json
from pathlib import Path

sentences = []
for file_name in os.listdir(text_path):
    if '.txt' in file_name: # and 'casos_clinicos_cardiologia98' in file_name:
        file_name_no_ext = file_name.rstrip('.txt')
                 
        file_path = Path(f'{text_path}/{file_name}')
        full_text = file_path.read_text()

        with open(f'{text_path}/{file_name}') as file:
            file_content = file.read()
            
        doc = nlp(file_content)
        
        print("Processing: ", file_name_no_ext)
        #T105	FARMACO 911 922	azatioprina

        if not is_test:
            annotations = pd.read_csv(f'{annotations_path}/{file_name_no_ext}.ann', sep='\t', 
                                      names=['ann_type', 'entity_type', 'text'], encoding='utf-8', keep_default_na=False)

            # drop annotator comments
            annotations.drop(annotations[annotations['ann_type'].str.startswith('#')].index, inplace=True)

            annotations['start_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[1]))
            annotations['end_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[2]))
            annotations['entity_type'] = annotations['entity_type'].apply(lambda v: v.split()[0])
            annotations.sort_values(by=['start_pos'], inplace=True, ignore_index=True)
        else:
            annotations = pd.DataFrame(columns=['ann_type', 'text'])
            
        total_chars = 0
        annotation_index = 0
        total_annotations = len(annotations)
        line_number = 0
        batch = []
        batch_number = 0
        for line in doc.sents:
            line_number += 1
            batch.append(line.text)

            if line_number % sents_per_batch == 0:
                batch_number += 1
                current_batch_annotation_indecies = []
                text = ''.join(batch)
                #max_index = total_chars + len(text) # - 1;
                #max_index = total_chars + full_text[total_chars:].find(text) + len(text) - 1
                batch_start = doc[line.start].idx #total_chars + full_text[total_chars:].find(text.strip()) #+ len(text) - 1
                batch_end = doc[line.start].idx + len(line.text) # batch_start + len(text) - 1
                #print(text, batch_start, batch_end)
                if batch_start < 0:
                    print('batch_start', full_text[total_chars:], text, total_chars)
                
                while annotation_index < total_annotations and annotations.iloc[annotation_index]['start_pos'] >= batch_start and annotations.iloc[annotation_index]['start_pos'] < batch_end:
                    annotation_text = str(annotations.iloc[annotation_index]['text'])
                    #print(annotation_text)
                    #new_start_pos = text.find(annotation_text)
                    new_start_pos = annotations.iloc[annotation_index]['start_pos'] - batch_start
                    
                    if new_start_pos < 0:
                        print('new_start_pos', text, annotation_text, batch_number, max_index)
                        
                    annotations.at[annotation_index, 'start_pos'] = new_start_pos
                    annotations.at[annotation_index, 'end_pos'] = new_start_pos + len(annotation_text)
                    current_batch_annotation_indecies.append(annotation_index)
                    annotation_index += 1

                format_batch_number = str(batch_number).zfill(3)
                with open(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{format_batch_number}-{batch_start}.txt', 'w', encoding='utf-8') as out_file:
                    out_file.write(text)

                # file_name_no_ext, batch_number, batch_start, text
                sentences.append({
                    'filename': file_name_no_ext,
                    'batch_number': batch_number,
                    'batch_start': batch_start,
                    'text': text
                })
                
                current_batch_annotations = annotations.iloc[current_batch_annotation_indecies,].copy().reset_index(drop=True)
                current_batch_annotations['entity_type_with_positions'] = None
                if not current_batch_annotations.empty:
                    current_batch_annotations['row_name'] = current_batch_annotations.apply(lambda row: row.name + 1, axis=1)
                    current_batch_annotations['ann_type'] = current_batch_annotations.apply(lambda row: f'{row["ann_type"].rstrip("0123456789")}{row.name + 1}', axis=1)
                    current_batch_annotations['entity_type_with_positions'] = current_batch_annotations.apply(lambda row: f'{row["entity_type"]} {row["start_pos"]} {row["end_pos"]}', axis=1)
                    current_batch_annotations.drop(columns=['start_pos', 'end_pos', 'entity_type'], inplace=True)

                current_batch_annotations.to_csv(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{format_batch_number}-{batch_start}.ann', sep='\t', header=False, index=False, columns=['ann_type', 'entity_type_with_positions', 'text'], encoding='utf-8')

                total_chars = batch_end #len(text) # -1
                batch.clear()
                
        if annotation_index < total_annotations:
            print('missing annotations')

Processing:  casos_clinicos_cardiologia10
Processing:  casos_clinicos_cardiologia102
Processing:  casos_clinicos_cardiologia108
Processing:  casos_clinicos_cardiologia111
Processing:  casos_clinicos_cardiologia112
Processing:  casos_clinicos_cardiologia115
Processing:  casos_clinicos_cardiologia120
Processing:  casos_clinicos_cardiologia125
Processing:  casos_clinicos_cardiologia128
Processing:  casos_clinicos_cardiologia129
Processing:  casos_clinicos_cardiologia13
Processing:  casos_clinicos_cardiologia132
Processing:  casos_clinicos_cardiologia134
Processing:  casos_clinicos_cardiologia135
Processing:  casos_clinicos_cardiologia136
Processing:  casos_clinicos_cardiologia138
Processing:  casos_clinicos_cardiologia139
Processing:  casos_clinicos_cardiologia140
Processing:  casos_clinicos_cardiologia141
Processing:  casos_clinicos_cardiologia143
Processing:  casos_clinicos_cardiologia144
Processing:  casos_clinicos_cardiologia146
Processing:  casos_clinicos_cardiologia148
Processing:  

Processing:  casos_clinicos_cardiologia499
Processing:  casos_clinicos_cardiologia50
Processing:  casos_clinicos_cardiologia500
Processing:  casos_clinicos_cardiologia502
Processing:  casos_clinicos_cardiologia505
Processing:  casos_clinicos_cardiologia509
Processing:  casos_clinicos_cardiologia51
Processing:  casos_clinicos_cardiologia511
Processing:  casos_clinicos_cardiologia515
Processing:  casos_clinicos_cardiologia519
Processing:  casos_clinicos_cardiologia520
Processing:  casos_clinicos_cardiologia526
Processing:  casos_clinicos_cardiologia531
Processing:  casos_clinicos_cardiologia532
Processing:  casos_clinicos_cardiologia533
Processing:  casos_clinicos_cardiologia534
Processing:  casos_clinicos_cardiologia538
Processing:  casos_clinicos_cardiologia542
Processing:  casos_clinicos_cardiologia543
Processing:  casos_clinicos_cardiologia546
Processing:  casos_clinicos_cardiologia548
Processing:  casos_clinicos_cardiologia549
Processing:  casos_clinicos_cardiologia550
Processing:  

In [26]:
df_sentences = pd.DataFrame.from_records(sentences)
df_sentences.tail()

,filename,batch_number,batch_start,text
17725,casos_clinicos_cardiologia98,95,9949,Medication:\n
17726,casos_clinicos_cardiologia98,96,9961,"Pancreatin 10,000 U: 3 tablets at breakfast, 3..."
17727,casos_clinicos_cardiologia98,97,10050,Continue with the usual regimen of novomix 26 ...
17728,casos_clinicos_cardiologia98,98,10149,Furosemide 40 mg 1 tablet at breakfast and hal...
17729,casos_clinicos_cardiologia98,99,10216,Bisoprolol 2.5 mg 1/2 tablet at breakfast.\n


In [27]:
df_sentences.shape

(17730, 4)

In [ ]:
df_sentences.to_csv(f'{output_path}/sentences.tsv', sep='\t', index=False)